In [14]:
import numpy as np
import argparse
import sys
import matplotlib.pyplot as plt
from drcstools.read_data import read_netcdfs
import xarray as xr
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import random_split
from torchmetrics import Metric
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger
from BalticRiverPrediction.BaltNet import AtmosphereDataModule
from BalticRiverPrediction.BaltNet import BaltNet, LightningModel
from BalticRiverPrediction.sharedUtilities import PredictionPlottingCallback
import torch
import torch.nn as nn

In [2]:
atmosphericForcing = "/data/meteo/BalticSea/RecentClimate/coastDat-3_COSMO-CLM/"
oderRunoff = "/silos/thomas/ModelSetup/OderBucht/oder_runoff.nc"

In [11]:
!ls {atmosphericForcing}/1961

clour.mom.dta.nc  pair.mom.dta.nc  shumi.mom.dta.nc  tairK.mom.dta.nc
lwdn2.mom.dta.nc  prec.mom.dta.nc  snow.mom.dta.nc   windx.mom.dta.nc
lwdn.mom.dta.nc   rain.mom.dta.nc  swdn.mom.dta.nc   windy.mom.dta.nc


In [3]:
L.seed_everything(123)
torch.set_float32_matmul_precision("medium")

Global seed set to 123


In [6]:
runoff = xr.open_dataset(f"{oderRunoff}").load()
runoff = runoff.sel(time=slice("1951", "2021"))
runoff = runoff.r_003

In [16]:
DataRain

<xarray.Dataset>
Dimensions:  (lon: 329, lat: 288, time: 32409)
Coordinates:
  * lon      (lon) float64 -5.0 -4.835 -4.67 -4.505 ... 35.92 36.09 36.25 36.41
  * lat      (lat) float64 48.0 48.08 48.16 48.25 ... 69.53 69.61 69.62 69.7
  * time     (time) object 1948-01-01 00:00:00 ... 2023-12-31 00:00:00
Data variables:
    rain     (time, lat, lon) float32 8.899e-07 1.419e-06 2.081e-06 ... 0.0 0.0
Attributes: (12/15)
    CDI:             Climate Data Interface version 2.0.5 (https://mpimet.mpg...
    Conventions:     CF-1.4
    source:          int2lm_131101_2.00_clm4, COSMO-CLM5.0_clm14_aerosol_gust...
    institution:     Helmholtz-Zentrum Geesthacht, Institute of Coastal Resea...
    title:           Europe 0.165x0.165
    project_id:      MarEns
    ...              ...
    contact:         http://coastmod.hzg.de
    references:      http://www.clm-community.eu/
    creation_date:   2019-05-24 01:55:26
    CDO:             Climate Data Operators version 2.0.5 (https://mpimet.mpg...
    NCO:             netCDF Operators version 5.1.1 (Homepage = http://nco.sf...
    history:         Wed Dec 14 14:54:40 2022: ncatted -a cartesian_axis,lon,...

In [17]:
test = xr.open_dataset(f"{atmosphericForcing}/1961/rain.mom.dta.nc")

In [25]:
test.isel(time=len(test.time)//2).time.dt.year

<xarray.DataArray 'year' ()>
array(1961)
Coordinates:
    time     object 1961-07-02 12:00:00
Attributes:
    standard_name:   time
    long_name:       time
    bounds:          time_bnds
    axis:            T
    cartesian_axis:  T

In [27]:
test.sel(time=str(test.isel(time=len(test.time)//2).time.dt.year.data))

<xarray.Dataset>
Dimensions:    (time: 8760, bnds: 2, lon: 252, lat: 264)
Coordinates:
  * time       (time) object 1961-01-01 00:00:00 ... 1961-12-31 23:00:00
  * lon        (lon) float64 -5.0 -4.835 -4.67 -4.505 ... 36.09 36.25 36.41
  * lat        (lat) float64 48.0 48.08 48.16 48.25 ... 69.45 69.53 69.61 69.7
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object ...
    rain       (time, lat, lon) float32 ...
Attributes: (12/15)
    CDI:             Climate Data Interface version 2.1.0 (https://mpimet.mpg...
    Conventions:     CF-1.4
    source:          int2lm_131101_2.00_clm4, COSMO-CLM5.0_clm14_aerosol_gust...
    institution:     Helmholtz-Zentrum Geesthacht, Institute of Coastal Resea...
    title:           Europe 0.165x0.165
    project_id:      MarEns
    ...              ...
    contact:         http://coastmod.hzg.de
    references:      http://www.clm-community.eu/
    creation_date:   2019-09-06 06:57:30
    CDO:             Climate Data Operators version 2.1.0 (https://mpimet.mpg...
    NCO:             netCDF Operators version 5.1.1 (Homepage = http://nco.sf...
    history:         Wed Dec 14 14:54:43 2022: ncatted -a cartesian_axis,lon,...

In [28]:
DataRain = read_netcdfs(
    f"{atmosphericForcing}/????/rain.mom.dta.nc",
    dim="time",
    transform_func=lambda ds:ds.sel(time=str(ds.isel(time=len(ds.time)//2).time.dt.year.data)).resample(time="1D").mean()
)
DataRain = DataRain.sel(time=slice("1951", "2021"))


  0%|          | 0/76 [00:00<?, ?it/s]

100%|██████████| 76/76 [10:37<00:00,  8.39s/it]


In [29]:
DataShumi = read_netcdfs(
    f"{atmosphericForcing}/????/shumi.mom.dta.nc",
    dim="time",
    transform_func=lambda ds:ds.sel(time=str(ds.isel(time=len(ds.time)//2).time.dt.year.data)).resample(time="1D").mean()
)
DataShumi = DataShumi.sel(time=slice("1951", "2021"))

100%|██████████| 76/76 [09:51<00:00,  7.78s/it]


In [30]:
DataWindSpeedx = read_netcdfs(
    f"{atmosphericForcing}/????/windx.mom.dta.nc",
    dim="time",
    transform_func=lambda ds:ds.sel(time=str(ds.isel(time=len(ds.time)//2).time.dt.year.data)).resample(time="1D").mean()
)
DataWindSpeedx = DataWindSpeedx.sel(time=slice("1951", "2021"))

100%|██████████| 76/76 [10:40<00:00,  8.42s/it]


In [31]:
DataWindSpeedy = read_netcdfs(
    f"{atmosphericForcing}/????/windy.mom.dta.nc",
    dim="time",
    transform_func=lambda ds:ds.sel(time=str(ds.isel(time=len(ds.time)//2).time.dt.year.data)).resample(time="1D").mean()
)
DataWindSpeedy = DataWindSpeedy.sel(time=slice("1951", "2021"))

 66%|██████▌   | 50/76 [09:52<06:43, 15.51s/it]

In [ ]:
DataTemp = read_netcdfs(
    f"{atmosphericForcing}/????/tairK.mom.dta.nc",
    dim="time",
    transform_func=lambda ds:ds.sel(time=str(ds.isel(time=len(ds.time)//2).time.dt.year.data)).resample(time="1D").mean()
)
DataTemp = DataTemp.sel(time=slice("1951", "2021"))

In [ ]:
assert DataShumi.time[0] == DataRain.time[0] == DataWindSpeed.time[0]
assert len(DataShumi.time) == len(DataRain.time) == len(DataWindSpeed.time)

data = xr.merge([DataRain, DataShumi, DataWindSpeed, DataTemp])
assert len(runoff.time) == len(data.time)